In [4]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math 

from more_itertools import distinct_permutations

### Highlight. 

In [ ]:
def highlight_correct_ticks(ax, EXACT_COVERS):
    """
    Highlights ticks on the axis corresponding to `EXACT_COVERS`.

    - If the state is an exact cover, the tick color is red.
    - If it is a minimum exact cover, the tick color is green.
    - If it matches a single "1" permutation, the tick color is light grey.
    - Otherwise, the tick color is black.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The axis object whose ticks will be modified.
    EXACT_COVERS : list of str
        A list of exact cover states, where each state is a binary string.

    Returns
    -------
    None
    """
    xlabels = [elem.get_text() for elem in ax.xaxis.get_ticklabels()]
    minimum_length = min([x.count("1")  for x in EXACT_COVERS])
    MEC = [state for state in EXACT_COVERS if state.count("1") == minimum_length][0]
    n = len(MEC)
    one_one_states = ["".join(elem) for elem in distinct_permutations('0'*(n-1) + '1')]
    
    for (state, ticklbl) in zip(xlabels, ax.xaxis.get_ticklabels()):
        ticklbl.set_color('limegreen' if state==MEC
                          else'crimson' if state in EXACT_COVERS
                          else 'grey' if state in one_one_states
                          else 'black')

### Underline.

In [12]:
def underline_states(ax, states_to_underline, fontsize):
    """
    Underlines specific states on the x-axis of a plot.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The axis object whose states will be underlined.
    states_to_underline : list of str
        A list of states (as strings) to underline.
    fontsize : int
        Font size for the underlines.

    Returns
    -------
    None
    """
    for elem in ax.xaxis.get_ticklabels():
        if elem.get_text() in states_to_underline:
            l = len(elem.get_text())
            # Aggiungi una distensione maggiore per la sottolineatura
            ax.annotate('_'*l, xy=(elem.get_position()[0], 0), 
                        xytext=(0, -10),  # Spostamento verso il basso
                        xycoords=('data', 'axes fraction'), 
                        textcoords='offset points',
                        ha='center', va='top', 
                        rotation=90, fontsize=fontsize, color='grey')

### Plot histograms from dataframe.

In [6]:
def plot_histogram_of_df_column(df, column_to_plot, EXACT_COVERS, states_to_underline, title=''):
    """
    Plots a histogram of a specified column from a dataframe, highlighting exact covers and underlining specific states.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with a 'states' index and numerical data.
    column_to_plot : str
        The column name to plot as a histogram.
    EXACT_COVERS : list of str
        A list of exact cover states to highlight.
    states_to_underline : list of str
        A list of states to underline on the x-axis.
    title : str, optional
        The title of the histogram plot (default is '').

    Returns
    -------
    matplotlib.axes.Axes
        The Axes object for the plot.
    """
    df = df.set_index('states')
    df = df.astype(float).fillna(0.0)
    
    # ##### COMPUTE PERCENTAGES
    total = df.sum()
    percentage = (df/ total) * 100

    percentage = percentage[[column_to_plot]]
    percentage = percentage.sort_values(column_to_plot, ascending=False)
    
    ##### FIGURE
    plt.figure(figsize=(10,5))
    N = 10
    ax = sns.barplot(x="states", y=column_to_plot, data=percentage, 
                     width=0.7, color='red', alpha=0.5) #label=f"Lowest energy among {random_attempts} random attempts"
    
    ### Make labels with percentages on the bars.
    labels = percentage[column_to_plot].round(1).astype('str') + '%'
    for container in ax.containers:
            ax.bar_label(container, labels=labels, fontsize=N-3)
            
    ### Highlight exact covers' ticks and underline initial states.
    df_for_ticks = percentage.copy()
    df_for_ticks["states"] = df_for_ticks.index
    underline_states(plt.gca(), states_to_underline, fontsize=N+2)
    highlight_correct_ticks(plt.gca(), EXACT_COVERS)
    
    ### Refinements.
    plt.xlabel("states", fontsize=N)
    plt.ylabel("", fontsize=N)
    plt.xticks(fontsize=N-2, rotation="vertical")
    plt.yticks(fontsize=N)
    plt.xlim(xmin=-1)
    plt.ylim(ymin=0, ymax=106)
    plt.minorticks_on()
    plt.grid(alpha=0.2)
    plt.title(title, fontsize=N)

    return ax        

In [ ]:
def plot_histogram_of_best_column(df, best_column, EXACT_COVERS, states_to_underline, title=''):
    """
    Plots the histogram of the best column from a dataframe, highlighting exact covers and underlining specific states.
    Also overlays error bars for the average values.

    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with a 'states' index and numerical data.
    best_column : str
        The column name considered the best and used for histogram values.
    EXACT_COVERS : list of str
        A list of exact cover states to highlight.
    states_to_underline : list of str
        A list of states to underline on the x-axis.
    title : str, optional
        The title of the histogram plot (default is '').

    Returns
    -------
    None
    """
    ax = plot_histogram_of_df_column(df, best_column, EXACT_COVERS, states_to_underline, title=title)
    
    df = df.set_index('states')
    df = df.astype(float).fillna(0.0)

    ##### COMPUTE PERCENTAGES AND ADD AVERAGE COLUMN
    total = df.sum()
    percentage = (df/ total) * 100
    
    percentage['average'] = percentage.mean(numeric_only=True, axis=1)
    percentage['std'] = percentage[percentage.columns[:-1]].std(numeric_only=True, axis=1)
        
    ##### KEEP THE BEST AND THE AVERAGE RESULT
    percentage = percentage[[best_column, "average", "std"]]
    percentage = percentage.sort_values(best_column, ascending=False)
    
            
    ### Display errors, but just for the "average" bars.
    x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches] 
    y_coords = percentage["average"]
    ax.errorbar(x=x_coords, y=y_coords, yerr=percentage["std"], linestyle="",
                markerfacecolor='none', linewidth=1,
                marker='o', color='k', ecolor='k', elinewidth=0.7, capsize=3.5, 
                barsabove=True, alpha=0.5) # label=f"Average on {random_attempts} random attempts"

    plt.show()

### Definisci i parametri del file leggendo il suo nome

In [7]:
def define_parameters_from_filename(filename, verbose):
    """
    Extracts parameters from a filename string.

    Parameters
    ----------
    filename : str
        The filename containing parameter details, such as "dim6_mail5_all0_random_p3_10ra_k0.085_".
    verbose : bool
        If True, prints the extracted parameters.

    Returns
    -------
    tuple
        A tuple containing:
        - n (int): Instance dimension.
        - instance (int): Instance number.
        - init_name (str): Initialization type.
        - p (int): Maximum layer or circuit depth.
        - ra (int): Number of random attempts.
        - k (float): Problem parameter (e.g., `k = l1/l2 * n`).

    Example
    -------
    >>> define_parameters_from_filename("dim6_mail3_all0_random_p3_10ra_k0.067_", verbose=True)
    n=6, i=3, init=all0, p=3, ra=10, k=0.067
    """
    filename = filename.rsplit('/', 1)[1] #remove path
    n = int(filename.split('dim')[1][0])
    instance = filename.split('mail')[1]
    instance = int(instance.split('_')[0])
    if "all" in filename:
        init_name = filename.split(f'mail{instance}_')[1]
        init_name = init_name.split("_")[0]
    else:
        init_name = 'customized'
    p = int(filename.split('_p')[1].split('_')[0])
    ra = filename.split(f'p{p}_')[1]
    ra = int(ra.split("ra")[0])
    k = filename.split("ra_k")[1]
    k = float(k.split("_")[0])
    

    if verbose:
        print("n={}, i={}, init={}, p={}, ra={}, k={}".format(n, instance, init_name, p, ra, k))
        
        
    return n, instance, init_name, p, ra, k

## Reading Random Parameters files

### Fai un elenco di tutti i file e datafile che contenogono una certa sottostringa.

In [2]:
def find_files_containing_string(strings, path, verbose=False):
    """
    Trova file che contengono specifiche sottostringhe nel nome e li ordina in base a un parametro estratto.

    Parametri:
        strings (list): Lista di sottostringhe da cercare nel nome dei file.
        path (str): Percorso della directory in cui cercare i file.

    Ritorna:
        tuple: Due liste di file ordinate:
               - FILENAME_list: File normali.
               - DATA_FILENAME_list: File con `_data` nel nome.
    """
    FILENAME_list = []
    DATA_FILENAME_list = []

    # Trova file che contengono tutte le sottostringhe in `strings`
    for obj in os.listdir(path):
        if os.path.isfile(os.path.join(path, obj)) and np.all([s in obj for s in strings]):
            if '_data' in obj:
                DATA_FILENAME_list.append(path + obj)
            else:
                FILENAME_list.append(path + obj)

    # Funzione per estrarre il parametro 'p' dal nome del file
    def extract_instance_from_filename(filename):
        _, instance, _, _, _, _ = define_parameters_from_filename(filename, verbose=False)
        return instance

    # Ordina le liste di file in base al valore di p estratto
    FILENAME_list.sort(key=lambda x: extract_instance_from_filename(x))
    DATA_FILENAME_list.sort(key=lambda x: extract_instance_from_filename(x))

    if verbose:
        for f,d in zip(FILENAME_list, DATA_FILENAME_list):
            print(f,"\n", d,"\n\n")
            
    return FILENAME_list, DATA_FILENAME_list


### Fai il plot una lista di file, datafile, sottolineando gli stati iniziali

In [1]:
from ipynb.fs.full.useful_functions_to_study_an_instance import define_instance, find_spectrum
import pandas as pd

def plot_file(FILENAME, DATA_FILENAME, colorchosen, alpha,
               states_to_underline=None, 
               title=None,
               dont_show_in_title=[], 
               pars=[], figsize=(18, 8), dpi=300, N=10):
    """
    """
    # Initialize the figure.
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(1,1,1)
    
    # Extract parameters from filename if not provided.
    if not pars:
        n, instance, init_name, p, random_attempts, k = define_parameters_from_filename(DATA_FILENAME, verbose=False)
    else:
        n, instance, init_name, p, random_attempts, k = pars

    # Define the problem instance.
    U, subsets_dict = define_instance(n, instance, verbose=False)

    # Analyze spectrum to extract relevant state information.
    states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k=1)
    MEC = min(EXACT_COVERS, key=lambda s: s.count('1'))
    
    # Load data into a pandas DataFrame.
    df = pd.read_csv(FILENAME, dtype=str).set_index('states')
    df = df.astype(float).fillna(0.0)

    # Compute percentages and add average and standard deviation columns.
    total = df.sum()
    percentage = (df / total) * 100
    percentage['average'] = percentage.mean(numeric_only=True, axis=1)
    percentage['std'] = percentage[percentage.columns[:-1]].std(numeric_only=True, axis=1)

    # Identify the best histogram index from the metadata file.
    with open(DATA_FILENAME, 'r') as DATA_FILE:
        for line in DATA_FILE:
            if 'Attempt that reached the best result with' in line:
                string = line.split('#')[1]
                i_best = string.split(' ')[0]

    # Select the column corresponding to the best result.
    column_best = f'counts_p{p}_{i_best}of{random_attempts}'

    # Keep the best and average results for plotting.
    percentage = percentage[[column_best, "average", "std"]]
    percentage = percentage.sort_values(column_best, ascending=False)

    ############################# LABELS ###################################
    # Create subplot.
    ax = sns.barplot(x="states", y=column_best, data=percentage, width=0.7, 
                     color=colorchosen, alpha=alpha)
    # Aggiungi il bordo alle barre
    for bar in ax.patches:
        bar.set_edgecolor('indigo')  # Colore del bordo

    # Annotate bars with percentage values.
    labels_df = percentage[column_best].round(1).astype(str).add('%')
    labels_df = labels_df.where(labels_df.index.isin(EXACT_COVERS), "")
          
    # for container in ax.containers:
    #     ax.bar_label(container, labels=labels_df.tolist(), fontsize=N-3)

    # Aggiungo le etichette percentuali manualmente, per poter regolare 
    # la loro x, che sarà allineata con quella della bar.
    for container in ax.containers:
        for rect, label in zip(container, labels_df.tolist()):
            label_x = rect.get_x()
            label_y = rect.get_height()
            ax.text(label_x, label_y, label, fontsize=N, ha='left', va='bottom')
                
        
    # Add error bars for average values.
    x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches]
    y_coords = percentage["average"]
    ax.errorbar(x=x_coords, y=y_coords, yerr=percentage["std"], linestyle="",
                markerfacecolor='none', linewidth=1, marker='o', color='k', ecolor='k', 
                elinewidth=0.7, capsize=3.5, barsabove=True, alpha=0.6)

    ########################### HIGHLIGHT #####################################
    # Highlight exact covers.
    df_for_ticks = percentage.copy()
    df_for_ticks["states"] = df_for_ticks.index
    highlight_correct_ticks(ax, EXACT_COVERS)

    # Sottolinea gli stati iniziali
    if init_name == "all1":
        one_one_states = ["".join(elem) for elem in distinct_permutations('0' * (n - 1) + '1')]
        init_string = "$|1 \\rangle$-initialization"
        underline_states(ax, one_one_states, fontsize=N+2)
    elif init_name == "all0":
        underline_states(ax, "0" * n, fontsize=N+2)
        init_string = "$|0 \\rangle$-initialization"
    else:
        underline_states(ax, list_of_states_to_underline, fontsize=N+4)
        init_string = ""

    ############################### TITLE #################################
    # Construct subplot title.
    dictstring = {"n":f"$n={n}$", "i":f"Instance #{instance}", "init":f"{init_string}", 
                  "p":f"$p={p}$", "ra":f"$ra={random_attempts}$", "k":f"$k={k}$"}
    title_string = ', '.join([dictstring[x] for x in dictstring if x not in dont_show_in_title])

    # Append bounds information to the title.
    bounds_and_pars0 = FILENAME.split('pars0')[1].split('.csv')[0]
    bounds_and_pars0 = bounds_and_pars0.replace("pi", "\pi").replace("x", "\\times")
    title_string += f"\n${bounds_and_pars0}$"

    ax.set_title(title_string, fontsize=N)
    
    ################################################################
    # Refine plot aesthetics.
    plt.xlabel("states", fontsize=N)
    plt.ylabel("percentage [%]", fontsize=N)
    plt.xticks(fontsize=N, rotation="vertical")
    plt.yticks(fontsize=N)
    plt.xlim(xmin=-1)
    plt.ylim(ymin=0, ymax=106)
    plt.minorticks_on()
    plt.grid(alpha=0.2)

    # Adjust subplot layout and display the figure.
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.8)
    plt.show()

In [2]:
from ipynb.fs.full.useful_functions_to_study_an_instance import define_instance, find_spectrum
import pandas as pd

def plot_list_of_files(FILENAME_list, DATA_FILENAME_list, colorchosen, alpha,
                       states_to_underline=None, 
                       title=None,
                       dont_show_in_titles=[], 
                       pars=[], figsize=(18, 8), dpi=300, N=10):
    """
    """
    # Initialize the figure.
    fig = plt.figure(figsize=figsize, dpi=dpi)

    # Set the overall title if provided.
    if title is not None:
        fig.suptitle(title, fontsize=N+3)

    # Determine the number of rows and columns for subplots.
    num_cols = math.ceil(len(FILENAME_list) / 2)
    num_rows = 2

    # Iterate through file pairs to create subplots.
    for num_subplot, (FILENAME, DATA_FILENAME) in enumerate(zip(FILENAME_list, DATA_FILENAME_list)):
        # Extract parameters from filename if not provided.
        if not pars:
            n, instance, init_name, p, random_attempts, k = define_parameters_from_filename(DATA_FILENAME, verbose=False)
        else:
            n, instance, init_name, p, random_attempts, k = pars

        # Define the problem instance.
        U, subsets_dict = define_instance(n, instance, verbose=False)

        # Analyze spectrum to extract relevant state information.
        states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k=1)
        MEC = min(EXACT_COVERS, key=lambda s: s.count('1'))
        
        # Load data into a pandas DataFrame.
        df = pd.read_csv(FILENAME, dtype=str).set_index('states')
        df = df.astype(float).fillna(0.0)

        # Compute percentages and add average and standard deviation columns.
        total = df.sum()
        percentage = (df / total) * 100
        percentage['average'] = percentage.mean(numeric_only=True, axis=1)
        percentage['std'] = percentage[percentage.columns[:-1]].std(numeric_only=True, axis=1)

        # Identify the best histogram index from the metadata file.
        with open(DATA_FILENAME, 'r') as DATA_FILE:
            for line in DATA_FILE:
                if 'Attempt that reached the best result with' in line:
                    string = line.split('#')[1]
                    i_best = string.split(' ')[0]

        # Select the column corresponding to the best result.
        column_best = f'counts_p{p}_{i_best}of{random_attempts}'

        # Keep the best and average results for plotting.
        percentage = percentage[[column_best, "average", "std"]]
        percentage = percentage.sort_values(column_best, ascending=False)

        ############################# LABELS ###################################
        # Create subplot.
        fig.add_subplot(num_rows, num_cols, num_subplot+1)
        ax = sns.barplot(x="states", y=column_best, data=percentage, width=0.7, 
                         color=colorchosen, alpha=alpha)
        # Aggiungi il bordo alle barre
        for bar in ax.patches:
            bar.set_edgecolor('indigo')  # Colore del bordo

        # Annotate bars with percentage values.
        labels_df = percentage[column_best].round(1).astype(str).add('%')
        labels_df = labels_df.where(labels_df.index.isin(EXACT_COVERS), "")
          
        # for container in ax.containers:
        #     ax.bar_label(container, labels=labels_df.tolist(), fontsize=N-3)

        # Aggiungo le etichette percentuali manualmente, per poter regolare 
        # la loro x, che sarà allineata con quella della bar.
        for container in ax.containers:
            for rect, label in zip(container, labels_df.tolist()):
                label_x = rect.get_x()
                label_y = rect.get_height()
                ax.text(label_x, label_y, label, fontsize=N, ha='left', va='bottom')
                
        
        # Add error bars for average values.
        x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches]
        y_coords = percentage["average"]
        ax.errorbar(x=x_coords, y=y_coords, yerr=percentage["std"], linestyle="",
                    markerfacecolor='none', linewidth=1, marker='o', color='k', ecolor='k', 
                    elinewidth=0.7, capsize=3.5, barsabove=True, alpha=0.6)

        ########################### HIGHLIGHT #####################################
        # Highlight exact covers.
        df_for_ticks = percentage.copy()
        df_for_ticks["states"] = df_for_ticks.index
        highlight_correct_ticks(ax, EXACT_COVERS)

        # Sottolinea gli stati iniziali
        if init_name == "all1":
            one_one_states = ["".join(elem) for elem in distinct_permutations('0' * (n - 1) + '1')]
            init_string = "$|1 \\rangle$-initialization"
            underline_states(ax, one_one_states, fontsize=N+2)
        elif init_name == "all0":
            underline_states(ax, "0" * n, fontsize=N+2)
            init_string = "$|0 \\rangle$-initialization"
        else:
            underline_states(ax, list_of_states_to_underline, fontsize=N+4)
            init_string = ""

        ############################### TITLE #################################
        # Construct subplot title.
        dictstring = {"n":f"$n={n}$", "i":f"Instance #{instance}", "init":f"{init_string}", 
                      "p":f"$p={p}$", "ra":f"$ra={random_attempts}$", "k":f"$k={k}$"}
        title_string = ', '.join([dictstring[x] for x in dictstring if x not in dont_show_in_titles])

        # Append bounds information to the title.
        bounds_and_pars0 = FILENAME.split('pars0')[1].split('.csv')[0]
        bounds_and_pars0 = bounds_and_pars0.replace("pi", "\pi").replace("x", "\\times")
        title_string += f"\n${bounds_and_pars0}$"

        ax.set_title(title_string, fontsize=N)
        
        ################################################################
        if num_subplot == 0 or num_subplot == 5: 
            ax.set_ylabel("percentage [%]")
            
        # Refine plot aesthetics.
        plt.xlabel("states", fontsize=N)
        plt.xticks(fontsize=N-1, rotation="vertical")
        plt.yticks(fontsize=N)
        plt.xlim(xmin=-1)
        plt.ylim(ymin=0, ymax=106)
        plt.minorticks_on()
        plt.grid(alpha=0.2)

    # Adjust subplot layout and display the figure.
    plt.subplots_adjust(wspace=0.17, hspace=0.6, left=0.03, right=0.97)
    plt.show()

## Reading Parameters Fixing files

In [2]:
# OK
def find_files_containing_string_parameters_fixing(directory, substring):
    """
    import os
    import csv
    
    import os
    import csv
    import re
    """

    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    files = [f for f in files if substring in f]

    
    datafiles = []
    for f in files:
        if f.endswith("_data.txt"):
            datafiles.append(f)
    
    # Funzione per estrarre il parametro 'p' dal nome del file
    def extract_instance_from_filename(filename):
        instance = int(filename.split("mail")[1].split("_")[0])
        return instance
    
    # Ordina le liste di file in base al valore di p estratto
    datafiles.sort(key=lambda x: extract_instance_from_filename(x))


    list_of_list_of_files = []
    for d in datafiles:
        header = d.split("_data")[0]
        list_of_files = []
        for f in files:
            if not f in datafiles and f.rsplit("_p", 1)[0] == header: # nota che rsplit parte dal fondo
                list_of_files.append(f)
        list_of_list_of_files.append(list_of_files)
        
    return datafiles, list_of_list_of_files

In [2]:
def plot_file_parameter_fixing(path, datafile, associated_files, init_name, title=None,
                               dont_show_in_title=[], figsize=(16, 10), N=10, dpi=300):
    """
    """
    
    # Crea la figura
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(1,1,1)


    ################################ ESTRAI PARAMETRI ################################
    n, instance, init_name, maxp, random_attempts, k = define_parameters_from_filename(path+datafile, verbose=False)
    plot_title = f"n={n}, i={instance}, init={init_name}, maxp={maxp}, ra={random_attempts}, k={k}"

    U, subsets_dict = define_instance(n, instance, verbose=False)
    states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k)
    
    ##################################################################################
    df_final = None

    # Per ogni file associato
    for file_to_read in associated_files:
        df = pd.read_csv(path+file_to_read, dtype={'states': "str"})

        if df_final is None:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, on="states", how="outer")

    # Riempimento e normalizzazione
    df_final = df_final.fillna(0)
    for p in range(1, maxp + 1):
        column_name = f'counts_p{p}'
        total = df_final[column_name].sum()
        if total > 0:
            df_final[column_name] = (df_final[column_name] / total) * 100

     # Ordina le colonne in base a "p" preservando l'ordine numerico
    sorted_columns = sorted(
        [col for col in df_final.columns if col.startswith('counts_p')],
        key=lambda x: int(x.split('p')[1])
    )

    ################################ PLOT ################################

    colors = ["darkorange", "crimson", "indigo"]
    df_final.plot(
        x='states', kind="bar", width=0.7, fontsize=N, stacked=False, ax=ax, legend=True,
        color=colors, alpha=1
    )

    ################################ Etichette sulle barre ###############################
    for state in df_final['states']:
        max_height = 0
        label_text = ""
        x_position = None

        for i, container in enumerate(ax.containers):
            p = i + 1
            row = df_final[df_final['states'] == state]
            if not row.empty:
                value = row[f'counts_p{p}'].values[0]
                if value > max_height:
                    max_height = value
                    label_text = f"{value:.1f}%"
                    x_position = container[df_final[df_final['states'] == state].index[0]].get_x() + container[0].get_width() / 2

        if max_height > 0:
            ax.text(
                x_position, max_height,
                label_text, ha='center', va='bottom', fontsize=N
                )

    ############################### EVIDENZIA E SOTTOLINEA ###############################
    highlight_correct_ticks(ax, EXACT_COVERS)

    # Sottolinea gli stati iniziali
    if init_name == "all1":
        one_one_states = ["".join(elem) for elem in distinct_permutations('0' * (n - 1) + '1')]
        init_string = "$|1 \\rangle$-initialization"
        underline_states(ax, one_one_states, fontsize=N+2)
    elif init_name == "all0":
        underline_states(ax, "0" * n, fontsize=N+2)
        init_string = "$|0 \\rangle$-initialization"
    else:
        underline_states(ax, list_of_states_to_underline, fontsize=N+4)
        init_string = ""


    ############################### TITLE #################################
    if title==None:
        # Construct subplot title.
        dictstring = {"n":f"$n={n}$", "i":f"Instance #{instance}", "init":f"{init_string}", 
                      "p":f"$p={p}$", "ra":f"$ra={random_attempts}$", "k":f"$k={k}$"}
        title_string = ', '.join([dictstring[x] for x in dictstring if x not in dont_show_in_title])
        
        # Append bounds information to the title.
        bounds_and_pars0 = datafile.split('pars0')[1].split('_data')[0]
        bounds_and_pars0 = bounds_and_pars0.replace("pi", "\pi").replace("x", "\\times")
        title_string += f"\n${bounds_and_pars0}$"
    
        ax.set_title(title_string, fontsize=N)
        
    else:
        ax.set_title(title, fontsize=N)
        
    ##################################################################
    
    ax.set_ylim(0, 103)
    ax.set_ylabel("percentage [%]")
    plt.minorticks_on()
    plt.grid(alpha=0.2)

    # Legenda
    legend = [f"layer {layer}" for layer in range(1, maxp+1)]
    ax.legend(legend)
    plt.show()

In [ ]:
# def plot_list_of_files_parameter_fixing(path, datafiles, associated_files, init_name, title=None,
#                        dont_show_in_titles=[],  figsize=(16, 10), N=10, dpi=300):
#     """
#     """
#     # Numero di subplot richiesti
#     n_subplots = len(datafiles)

#     # Imposta il numero di righe e colonne per i subplot
#     n_rows = 2
#     n_cols = (n_subplots + 1) // n_rows

#     # Crea la figura
#     fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize, dpi=dpi, squeeze=False)
#     fig.suptitle(title, fontsize=N + 2)

#     # Flatten dell'array di assi per accesso sequenziale
#     axes = axes.flatten()

#     # Loop attraverso ogni datafile e i suoi file associati
#     for idx, (datafile, associated_files) in enumerate(zip(datafiles, associated_files)):

#         ax = axes[idx]

#         ################################ ESTRAI PARAMETRI ################################
#         n, instance, init_name, maxp, random_attempts, k = define_parameters_from_filename(path+datafile, verbose=False)
#         plot_title = f"n={n}, i={instance}, init={init_name}, maxp={maxp}, ra={random_attempts}, k={k}"

#         U, subsets_dict = define_instance(n, instance, verbose=False)
#         states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k)
        
#         ##################################################################################
#         df_final = None

#         # Per ogni file associato
#         for file_to_read in associated_files:
#             file_to_read = path + file_to_read
#             df = pd.read_csv(file_to_read, dtype={'states': "str"})

#             if df_final is None:
#                 df_final = df
#             else:
#                 df_final = pd.merge(df_final, df, on="states", how="outer")

#         # Riempimento e normalizzazione
#         df_final = df_final.fillna(0)
#         for p in range(1, maxp + 1):
#             column_name = f'counts_p{p}'
#             total = df_final[column_name].sum()
#             if total > 0:
#                 df_final[column_name] = (df_final[column_name] / total) * 100

#          # Ordina le colonne in base a "p" preservando l'ordine numerico
#         sorted_columns = sorted(
#             [col for col in df_final.columns if col.startswith('counts_p')],
#             key=lambda x: int(x.split('p')[1])
#         )

#         ################################ PLOT ################################
#         ax.set_title(plot_title, fontsize=N)

#         colors = ["darkorange", "crimson", "indigo"]
#         df_final.plot(
#             x='states', kind="bar", width=0.7, fontsize=N, stacked=False, ax=ax, legend=True,
#             color=colors, alpha=1
#         )

#         ################################ Etichette sulle barre ###############################
#         for state in df_final['states']:
#             max_height = 0
#             label_text = ""
#             x_position = None

#             for i, container in enumerate(ax.containers):
#                 p = i + 1
#                 row = df_final[df_final['states'] == state]
#                 if not row.empty:
#                     value = row[f'counts_p{p}'].values[0]
#                     if value > max_height:
#                         max_height = value
#                         label_text = f"{value:.1f}%"
#                         x_position = container[df_final[df_final['states'] == state].index[0]].get_x() + container[0].get_width() / 2

#             if max_height > 0:
#                 ax.text(
#                     x_position, max_height,
#                     label_text, ha='center', va='bottom', fontsize=8, weight='bold'
#                 )

#         ############################### EVIDENZIA E SOTTOLINEA ###############################
#         highlight_correct_ticks(ax, EXACT_COVERS)

#         # Sottolinea gli stati iniziali
#         if init_name == "all1":
#             one_one_states = ["".join(elem) for elem in distinct_permutations('0' * (n - 1) + '1')]
#             init_string = "$|1 \\rangle$-initialization"
#             underline_states(ax, one_one_states, fontsize=N+2)
#         elif init_name == "all0":
#             underline_states(ax, "0" * n, fontsize=N+2)
#             init_string = "$|0 \\rangle$-initialization"
#         else:
#             underline_states(ax, list_of_states_to_underline, fontsize=N+4)
#             init_string = ""


#         ############################### TITLE #################################
#         # Construct subplot title.
#         dictstring = {"n":f"$n={n}$", "i":f"Instance \#{instance}", "init":f"{init_string}", 
#                       "p":f"$p={p}$", "ra":f"$ra={random_attempts}$", "k":f"$k={k}$"}
#         title_string = ', '.join([dictstring[x] for x in dictstring if x not in dont_show_in_titles])
        
#         # Append bounds information to the title.
#         bounds_and_pars0 = datafile.split('pars0')[1].split('_data')[0]
#         bounds_and_pars0 = bounds_and_pars0.replace("pi", "\pi").replace("x", "\\times")
#         title_string += f"\n${bounds_and_pars0}$"

#         ax.set_title(title_string, fontsize=N)
#         ##################################################################
        
#         ax.set_ylim(0, 103)
#         plt.minorticks_on()
#         plt.grid(alpha=0.2)

#         if idx == 0 or idx == 5: 
#             ax.set_ylabel("percentage [%]")
#         else:
#             ax.set_ylabel("")

#         # Legenda
#         legend = [f"layer {layer}" for layer in range(1, maxp+1)]
#         ax.legend(legend)


#     # Nascondi gli assi inutilizzati
#     for idx in range(n_subplots, len(axes)):
#         fig.delaxes(axes[idx])
    
#     plt.subplots_adjust(wspace=0.17, hspace=0.5, left=0.03, right=0.97)
#     plt.show()